# library

In [1]:
import pickle
from tqdm.auto import tqdm

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, BertTokenizerFast

# load data

In [2]:
with open('/content/drive/MyDrive/train_data.pickle', 'rb') as f:

    data = pickle.load(f)

In [3]:
print(len(data))

288230


In [ ]:
data[4]

{'knowledge': ['처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다.',
  '이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.'],
 'query': '그럼 이제 슬슬 시작해볼까봐요. 다들 먹이는 양이 다르던데 어느정도 먹이면 좋을까요?',
 'answer': '처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 거로 충분해요. 모유나 분유 외의 것을 처음 접하는 것이기 때문에 처음 먹는 날은 모유나 분유를 조금 먹인 다음 쌀미음 반작은술 정도 먹이고 다시 모유나 분유를 먹여보세요.'}

# preprocess

In [ ]:
d = data[4]

In [ ]:
knowledge = ' '.join(d['knowledge'])

'처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다. 이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.'

In [ ]:
s = f"질문: {d['query']}\n지식: {knowledge}\n대답: "

print(s)

질문: 그럼 이제 슬슬 시작해볼까봐요. 다들 먹이는 양이 다르던데 어느정도 먹이면 좋을까요?
지식: 처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다. 이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.
대답: 


In [ ]:
o = d['answer']

print(o)

처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 거로 충분해요. 모유나 분유 외의 것을 처음 접하는 것이기 때문에 처음 먹는 날은 모유나 분유를 조금 먹인 다음 쌀미음 반작은술 정도 먹이고 다시 모유나 분유를 먹여보세요.


In [ ]:
f"지식: {' '.join(data[0]['knowledge'])}"

'지식: '

In [4]:
def preprocess(data):

    preprocessed_data = []

    for i in range(len(data)):

        d = data[i]

        knowledge = ' '.join(d['knowledge'])

        s = f"질문: {d['query']}\n지식: {knowledge}\n대답: {d['answer']}"

        preprocessed_data.append((i,s))

    return preprocessed_data

In [5]:
preprocessed_train_data = preprocess(data)

print(len(preprocessed_train_data))

288230


In [ ]:
preprocessed_train_data[0]

(0,
 '질문: 저희 애가 슬슬 이유식을 시작해야 할 것 같은데 언제 시작하면 좋을 지 모르겠어요.\n지식: \n대답: 아기가 지금 몇 개월이나 됐죠?')

# dataset

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('byeongal/Ko-DialoGPT')

In [6]:
tokenizer = BertTokenizerFast.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/344k [00:00<?, ?B/s]

In [38]:
s = tokenizer(preprocessed_train_data[2][1],max_length = 64, truncation = True, stride = 32, padding = 'max_length', return_overflowing_tokens=True)['input_ids']

In [ ]:
s

In [39]:
d = tokenizer(preprocessed_train_data[0][1])

print(d['input_ids'])

[2, 18798, 2028, 14325, 28830, 21370, 30159, 22154, 8311, 7657, 2190, 16023, 14599, 24072, 14389, 6266, 19984, 2016, 18508, 2028, 17862, 2028, 17818, 14136, 4145, 28840, 14030, 3285, 8290, 2033, 3]


In [ ]:
tokenizer.decode(3)

'[SEP]'

In [ ]:
labels = torch.tensor(d['input_ids']).clone()

In [ ]:
labels = torch.roll(labels,-1,-1)

In [ ]:
labels[-1] = -100

In [ ]:
labels[:-1] = -100

In [ ]:
train_dataset[0]

(tensor([    2, 18798,  2028, 14325, 28830, 21370, 30159, 22154,  8311,  7657,
          2190, 16023, 14599, 24072, 14389,  6266, 19984,  2016, 18508,  2028,
         17862,  2028, 17818, 14136,  4145, 28840, 14030,  3285,  8290,  2033,
             3]),
 tensor([18798,  2028, 14325, 28830, 21370, 30159, 22154,  8311,  7657,  2190,
         16023, 14599, 24072, 14389,  6266, 19984,  2016, 18508,  2028, 17862,
          2028, 17818, 14136,  4145, 28840, 14030,  3285,  8290,  2033,     3,
          -100]))

In [ ]:
tokenizer(preprocessed_train_data[0][1], max_length = 64, truncation = True, padding = "max_length")

{'input_ids': [2, 18798, 2028, 14325, 28830, 21370, 30159, 22154, 8311, 7657, 2190, 16023, 14599, 24072, 14389, 6266, 19984, 2016, 18508, 2028, 17862, 2028, 17818, 14136, 4145, 28840, 14030, 3285, 8290, 2033, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [ ]:
tokenizer.encode('[PAD]')

[2, 0, 3]

In [ ]:
#max length를 맞추지 않으면 batch를 못만든다

max_length = 0

for data in preprocessed_train_data:

    v = len(tokenizer(data[1])['input_ids'])

    if max_length < v:

        max_length = v

print(max_length)

548


In [7]:
class ChatbotDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length, stride):

        tokenizer = tokenizer

        self.data = []
        self.label = []

        for i in tqdm(range(len(dataset))):

            input_ids = tokenizer(dataset[i][1], max_length = max_length, stride = stride, padding = 'max_length', truncation = True, return_overflowing_tokens=True)['input_ids']

            for inputs in input_ids:

                inputs = torch.tensor(inputs)
                labels = inputs.clone()
                labels = torch.roll(labels,-1,-1)
                labels[-1] = -100

                self.data.append(inputs)
                self.label.append(labels)

    def __getitem__(self, i):
        return (self.data[i],self.label[i])

    def __len__(self):
        return len(self.label)

In [8]:
max_length = 32
stride = 16

train_dataset = ChatbotDataset(preprocessed_train_data,tokenizer, max_length = max_length, stride = stride)

  0%|          | 0/288230 [00:00<?, ?it/s]

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle = True)

# training

In [10]:
model = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0") if torch.cuda.is_available else torch.device('cpu')
model = model.to(device)

config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [9]:
device

device(type='cuda', index=0)

In [46]:
for data in train_dataloader:

    print(data)
    break

[tensor([[    2, 18798,  2028,  ...,  2033,     3,     0],
        [    2, 18798,  2028,  ...,     3,     0,     0],
        [    2, 18798,  2028,  ..., 25492, 32075,     3],
        ...,
        [    2,  2028,  5257,  ..., 21207, 19565,     3],
        [    2,  8051, 18464,  ..., 40908, 30159,     3],
        [    2, 21207, 19565,  ..., 14336, 24653,     3]]), tensor([[18798,  2028, 14325,  ...,     3,     0,  -100],
        [18798,  2028, 17818,  ...,     0,     0,  -100],
        [18798,  2028, 18173,  ..., 32075,     3,  -100],
        ...,
        [ 2028,  5257, 34148,  ..., 19565,     3,  -100],
        [ 8051, 18464,  8010,  ..., 30159,     3,  -100],
        [21207, 19565,  8155,  ..., 24653,     3,  -100]])]


In [47]:
logits = model(data[0])

In [ ]:
logits.logits

In [49]:
criterion(logits.logits.view(-1,42000), data[1].view(-1))

tensor(5.2101, grad_fn=<NllLossBackward0>)

In [ ]:
data[1].shape

torch.Size([4, 548])

In [ ]:
logits.logits.view(-1,42000).shape

torch.Size([2192, 42000])

In [ ]:
logits.logits.shape

torch.Size([4, 548, 42000])

In [ ]:
data[1].view(-1).shape

torch.Size([2192])

In [ ]:
model.train()

total_loss = 0.0
total_iter = 0
epochs = 1

for epoch in tqdm(range(epochs)):

    for i,batch in tqdm(enumerate(train_dataloader)):

        input_ids = batch[0].to(device)
        labels = batch[1].to(device)

        logits = model(input_ids).logits

        loss = criterion(logits.view(-1, 42000), labels.view(-1))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        total_iter += 1

        if total_iter % 1000 == 0:

            mean_loss = total_loss / total_iter
            print(f"epoch {epoch+1} : loss {mean_loss:1.4f}")


  0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

epoch 1 : loss 3.8552
epoch 1 : loss 3.7906
epoch 1 : loss 3.7576
epoch 1 : loss 3.7303
epoch 1 : loss 3.7098
epoch 1 : loss 3.6922
epoch 1 : loss 3.6771
epoch 1 : loss 3.6645
epoch 1 : loss 3.6541
epoch 1 : loss 3.6433
epoch 1 : loss 3.6318
epoch 1 : loss 3.6219
epoch 1 : loss 3.6125
epoch 1 : loss 3.6040
epoch 1 : loss 3.5958
epoch 1 : loss 3.5880
epoch 1 : loss 3.5803
epoch 1 : loss 3.5733
epoch 1 : loss 3.5666
epoch 1 : loss 3.5601
epoch 1 : loss 3.5537
epoch 1 : loss 3.5488
epoch 1 : loss 3.5423
epoch 1 : loss 3.5361
epoch 1 : loss 3.5301
epoch 1 : loss 3.5246
epoch 1 : loss 3.5192
epoch 1 : loss 3.5141
epoch 1 : loss 3.5090
epoch 1 : loss 3.5048
epoch 1 : loss 3.4997
epoch 1 : loss 3.4946
epoch 1 : loss 3.4897
epoch 1 : loss 3.4853
epoch 1 : loss 3.4810
epoch 1 : loss 3.4766
epoch 1 : loss 3.4722
epoch 1 : loss 3.4680
epoch 1 : loss 3.4643
epoch 1 : loss 3.4601
epoch 1 : loss 3.4559
epoch 1 : loss 3.4520
epoch 1 : loss 3.4482
epoch 1 : loss 3.4444
epoch 1 : loss 3.4407
epoch 1 : 

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
resume = "model.pth"

checkpoint = torch.load(resume, map_location=torch.device('cpu'))

model.load_state_dict(checkpoint)

# inference test

In [ ]:
model = GPT2LMHeadModel.from_pretrained('byeongal/Ko-DialoGPT')

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
model = GPT2LMHeadModel.from_pretrained("kykim/gpt3-kor-small_based_on_gpt2")

config.json:   0%|          | 0.00/621 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/526M [00:00<?, ?B/s]

In [ ]:
train_dataset[4]

(tensor([    2, 18798,  2028, 14716, 14130, 21370, 22154, 18200, 15867,  2016,
         15367, 31241, 16110, 15287, 16264, 17649, 24184,  8043, 31152,  8055,
          2033, 18508,  2028, 14121, 30159, 24072,  7653,  3118, 33424, 17688,
          7653,  4414, 31241, 14212, 40060,  2016, 30159, 14121, 14501, 20796,
         21098,  8155, 34199, 14103,  4090,  8159,  7876,  2019, 20121,  8133,
          8008,  5257, 34148,  4349, 14061, 15511,  4090,  8159, 14153, 14123,
         21098,  8155, 34199,  4090, 14872,  2016, 17862,  2028, 14121, 30159,
         24072,  7653,  3118, 33424, 17688,  7653,  4414, 31241,  2173,  8054,
         14676, 14088,  2016, 21098,  8155, 19285, 34538, 14199, 14121, 39684,
         27231, 14045, 14121, 14501, 20796, 21098,  8155, 34199, 14103,  4090,
          8159, 14153,  5257,  8272,  8309,  4349, 20121,  8133, 14061, 24653,
         14123, 21098,  8155, 34199, 21150, 16299,  2016,     3]),
 tensor([18798,  2028, 14716, 14130, 21370, 22154, 18200, 15867,

In [ ]:
logits = model(train_dataset[0][0]).logits

In [ ]:
logits

tensor([[-8.3233e+00, -1.5567e+00, -7.6835e+00,  ..., -2.8947e+00,
         -4.1576e+00, -4.6250e+00],
        [-7.3123e+00, -1.1532e+00, -6.8052e+00,  ..., -4.4670e+00,
         -2.0870e-01, -3.0655e+00],
        [-9.1188e+00,  9.6088e-01, -8.7226e+00,  ..., -2.4069e+00,
         -2.8122e+00, -7.2890e+00],
        ...,
        [-8.9656e+00, -8.1959e-01, -8.2295e+00,  ..., -4.6181e-01,
         -6.9682e-01, -5.2171e+00],
        [-1.1117e+01,  6.5995e-01, -1.0513e+01,  ...,  9.7575e-03,
         -2.1954e+00, -8.1042e+00],
        [-9.4697e+00,  1.7928e+00, -9.0498e+00,  ..., -2.5771e-02,
         -1.4134e+00, -8.8300e+00]], grad_fn=<MmBackward0>)

In [ ]:
train_dataset[0][0].shape

torch.Size([31])

In [ ]:
logits.shape

torch.Size([31, 42000])

In [ ]:
pred = logits.detach().numpy()

print(pred)

sorted_pred = torch.argmax(logits, axis = 1)

[[-8.3232775e+00 -1.5566703e+00 -7.6834588e+00 ... -2.8947427e+00
  -4.1575756e+00 -4.6250234e+00]
 [-7.3123364e+00 -1.1532438e+00 -6.8052378e+00 ... -4.4669981e+00
  -2.0870268e-01 -3.0655265e+00]
 [-9.1188450e+00  9.6087921e-01 -8.7226315e+00 ... -2.4069428e+00
  -2.8121502e+00 -7.2890267e+00]
 ...
 [-8.9655647e+00 -8.1958753e-01 -8.2295427e+00 ... -4.6181309e-01
  -6.9681716e-01 -5.2170935e+00]
 [-1.1117498e+01  6.5994829e-01 -1.0512745e+01 ...  9.7574741e-03
  -2.1953628e+00 -8.1042204e+00]
 [-9.4696941e+00  1.7928430e+00 -9.0497894e+00 ... -2.5770903e-02
  -1.4133973e+00 -8.8299828e+00]]


In [ ]:
sorted_pred

tensor([ 2016, 13990,     3,  6282, 14136, 17300, 15179,  8311, 14696, 16757,
        15510,  2016, 24262, 31152,  2190,  2016,  2016,     3,  8159, 17873,
         2028, 17873, 30159, 30159, 18889,  8008, 22520, 14526,  2033, 19246,
        18798])

In [ ]:
tokenizer.decode(sorted_pred)

'.했다 [SEP] 집 지금 커서 먹기야 하는데 때가 같아서. 시작할 좋을까 것.. [SEP]인 이유식 : 이유식 이유식을 이유식을 살이에 지났나요? 답변 질문'

In [ ]:
t = torch.tensor(train_dataset[4][0]['input_ids']).unsqueeze(0)

t.shape

torch.Size([1, 78])

In [ ]:
o = model.generate(t,max_length = 100)

In [ ]:
o.tolist()[0]

In [ ]:
print(tokenizer.decode(o.tolist()[0]))

질문: 그럼 이제 슬슬 시작해볼까봐요. 다들 먹이는 양이 다르던데 어느정도 먹이면 좋을까요?
지식: 처음 이유식을 시작하면 한 달간은 하루에 한 번 먹이는 것으로 충분하다. 이유식을 처음 먹는 날은 모유나 분유를 조금 먹인 후 1작은술에 쌀죽을 반 정도 담아 먹인 다음 다시 모유나 분유를 먹인다.
대답: 맵기 조절을 위한 방법 및 흡수율에 따라 조절해서 먹는 방법 및 흡수율에 따라 조절해서
